# Imports

In [1]:
import time

import pandas as pd


from cyclops.processors.cleaning import normalize_names, normalize_values
from cyclops.processors.column_names import (
    DISCHARGE_TIMESTAMP,
    ENCOUNTER_ID,
    EVENT_NAME,
    EVENT_VALUE,
)
from cyclops.processors.feature.split import intersect_datasets
from cyclops.utils.file import join, save_dataframe
from use_cases.gemini.wanglab.constants import (
    CLEANED_DIR,
    ENCOUNTERS_FILE,
    OUTCOME_DEATH,
    OUTCOME_EDEMA,
    QUERIED_DIR,
    TARGET_TIMESTAMP,
)
from use_cases.gemini.wanglab.query import main

2022-10-11 12:30:10,098 INFO cyclops.orm     - Database setup, ready to run queries!


# Query

In [ ]:
t = time.time()
cohort, events = main()
print(time.time() - t)
cohort

2022-10-11 12:30:16,102 INFO cyclops.orm     - Query returned successfully!
2022-10-11 12:30:16,103 INFO cyclops.utils.profile - Finished executing function run_query in 5.925108 s
2022-10-11 12:30:18,492 INFO cyclops.orm     - Query returned successfully!
2022-10-11 12:30:18,493 INFO cyclops.utils.profile - Finished executing function run_query in 2.388987 s
2022-10-11 12:30:30,615 INFO cyclops.orm     - Query returned successfully!
2022-10-11 12:30:30,617 INFO cyclops.utils.profile - Finished executing function run_query in 11.288929 s
2022-10-11 12:30:44,765 INFO cyclops.orm     - Query returned successfully!
2022-10-11 12:30:44,767 INFO cyclops.utils.profile - Finished executing function run_query in 2.112695 s
2022-10-11 12:30:45,597 INFO cyclops.orm     - Query returned successfully!
2022-10-11 12:30:45,598 INFO cyclops.utils.profile - Finished executing function run_query in 0.162419 s
2022-10-11 12:31:31,683 INFO cyclops.orm     - Query returned successfully!
2022-10-11 12:31:3

In [ ]:
cohort[OUTCOME_DEATH].sum() / len(cohort)

In [ ]:
cohort[OUTCOME_EDEMA].sum() / len(cohort)

In [ ]:
events

In [ ]:
# Intersect over encounter IDs to get only those encounters common to both
cohort, events = intersect_datasets([cohort, events], ENCOUNTER_ID)

In [ ]:
save_dataframe(events, join(QUERIED_DIR, "batch_0000.parquet"))

# Clean / Preprocess

In [ ]:
death_events = cohort[cohort[OUTCOME_DEATH] == True]  # noqa: E712
death_events = death_events[[ENCOUNTER_ID, DISCHARGE_TIMESTAMP]]
death_events = death_events.rename({DISCHARGE_TIMESTAMP: TARGET_TIMESTAMP}, axis=1)
cohort = pd.merge(cohort, death_events, on=ENCOUNTER_ID, how="left")
cohort

In [ ]:
save_dataframe(cohort, ENCOUNTERS_FILE)

In [ ]:
# Normalize names and string values
events[EVENT_NAME] = normalize_names(events[EVENT_NAME])
events[EVENT_VALUE] = normalize_values(events[EVENT_VALUE])

# Convert values to numeric, dropping those which can't be converted
events[EVENT_VALUE] = pd.to_numeric(events[EVENT_VALUE], errors="coerce")
print("Length before:", len(events))
events = events[~events[EVENT_VALUE].isna()]
print("Length after:", len(events))
events

In [ ]:
save_dataframe(events, join(CLEANED_DIR, "batch_0000.parquet"))